In [2]:
# Loading modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Fontconfig warning: ignoring UTF-8: not a valid region tag
Matplotlib is building the font cache; this may take a moment.


In [ ]:
# Loading data
tree_table = pd.read_csv('table.dat', sep = ',', header=None, names=['Parent', 'Child'])
branch_lengths = pd.read_csv('branchlength.dat', sep = ',', header=None)
branch_lengths = pd.DataFrame(branch_lengths.values.flatten(), columns=['Length'])
msa = pd.read_csv('msa.dat', sep = ' ', header=None)
merged_data = pd.concat([tree_table, branch_lengths], axis=1)
merged_data['Sequence'] = np.nan
merged_data['Sequence'] = merged_data['Sequence'].astype(object)
# Iterate over msa and update the 'sequence' column in merged_data
for index, row in msa.iterrows():
    child_value = row[0]
    sequence_value = ' '.join(map(str, row[1:]))
    merged_data.loc[merged_data['Child'] == child_value, 'Sequence'] = sequence_value

merged_data

,Parent,Child,Length,Sequence
0,9,1,0.10,AGATCAAGATCAAGATCAAGATCAAGATCA
1,9,2,0.40,AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
2,8,9,0.01,NaN
3,8,3,0.04,CGCTATCGCTATCGCTATCGCTATCGCTAT
4,7,4,0.20,CGTTACCGTTACCGTTACCGTTACCGTTAC
5,7,5,0.08,CGCTACCGCTACCGCTACCGCTACCGCTAC
6,6,7,0.12,NaN
7,6,8,0.14,NaN


In [ ]:
# Creating the node class
class Node:

    def __init__(self, identity, parent=None, branch_length=None, sequence=None):
        self.identity = identity
        self.parent = parent
        self.branch_length = branch_length
        self.sequence = sequence
        self.children = []


Empty Tree
